In [8]:
import cobra
import json

In [55]:
def get_enzyme_constraint_model(json_model_file):
    """using enzyme concentration constraint
    json model to create a COBRApy model.

    Arguments
    ----------
    * json_model_file: json Model file.

    :return: Construct an enzyme-constrained model.
    """

    dictionary_model = json_load(json_model_file)
    model = cobra.io.json.load_json_model(json_model_file)
    prot_pool = cobra.Metabolite('prot_pool')
    prot_pool.compartment = 'c'
    coefficients = dict()
    for rxn in model.reactions:
        for eachr in dictionary_model['reactions']:
            if rxn.id == eachr['id']:
                if eachr['kcat_MW']:
                    rxn.add_metabolites({prot_pool:-1/float(eachr['kcat_MW'])})
                    # coefficients[rxn.forward_variable] = 1 / float(eachr['kcat_MW'])
                break

    lowerbound = dictionary_model['enzyme_constraint']['lowerbound']
    upperbound = dictionary_model['enzyme_constraint']['upperbound']
    
    R_pool = cobra.Reaction('enzyme_pool')
    R_pool.add_metabolites({prot_pool:1})
    R_pool.lower_bound = lowerbound
    R_pool.upper_bound = upperbound
    model.add_reactions([R_pool])
    #print(upperbound)
    # constraint = model.problem.Constraint(0, lb=lowerbound, ub=upperbound)
    # model.add_cons_vars(constraint)
    # model.solver.update()
    # constraint.set_linear_coefficients(coefficients=coefficients)
    return model
def get_enzyme_constraint_model_original(json_model_file):
    """using enzyme concentration constraint
    json model to create a COBRApy model.

    Arguments
    ----------
    * json_model_file: json Model file.

    :return: Construct an enzyme-constrained model.
    """

    dictionary_model = json_load(json_model_file)
    model = cobra.io.json.load_json_model(json_model_file)
    # prot_pool = cobra.Metabolite('prot_pool')
    # prot_pool.compartment = 'c'
    coefficients = dict()
    for rxn in model.reactions:
        for eachr in dictionary_model['reactions']:
            if rxn.id == eachr['id']:
                if eachr['kcat_MW']:
                    # rxn.add_metabolites({prot_pool:-1/float(eachr['kcat_MW'])})
                    coefficients[rxn.forward_variable] = 1 / float(eachr['kcat_MW'])
                break

    lowerbound = dictionary_model['enzyme_constraint']['lowerbound']
    upperbound = dictionary_model['enzyme_constraint']['upperbound']
    
    # R_pool = cobra.Reaction('enzyme_pool')
    # R_pool.add_metabolites({prot_pool:1})
    # R_pool.lower_bound = lowerbound
    # R_pool.upper_bound = upperbound
    # model.add_reactions([R_pool])
    #print(upperbound)
    constraint = model.problem.Constraint(0, lb=lowerbound, ub=upperbound)
    model.add_cons_vars(constraint)
    model.solver.update()
    constraint.set_linear_coefficients(coefficients=coefficients)
    return model

def json_load(path):
    """Loads the given JSON file and returns it as dictionary.

    Arguments
    ----------
    * path: The path of the JSON file
    """
    with open(path) as f:
        dictionary = json.load(f)
    return dictionary

In [41]:
dictionary_model = json_load('iBsu1147_irr_enz_constraint_adj.json')

In [43]:
dictionary_model['enzyme_constraint']

{'enzyme_mass_fraction': 0.588,
 'total_protein_fraction': 0.56,
 'average_saturation': 0.5,
 'lowerbound': 0,
 'upperbound': 0.165}

In [103]:
model = get_enzyme_constraint_model_original('iBsu1147_irr_enz_constraint_adj.json')


Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e_reverse,0.001825,0,0.00%
fe3_e,EX_fe3_e_reverse,0.001964,0,0.00%
glc__D_e,EX_glc__D_e,7.033,6,58.45%
glc__D_e,EX_glc__D_e_reverse,5,6,41.55%
k_e,EX_k_e_reverse,0.4024,0,0.00%
mg2_e,EX_mg2_e_reverse,0.05798,0,0.00%
nh4_e,EX_nh4_e_reverse,5.517,0,0.00%
o2_e,EX_o2_e,17,0,0.00%
pi_e,EX_pi_e_reverse,10.02,0,0.00%
so4_e,EX_so4_e_reverse,0.1213,0,0.00%


# Fix model

In [118]:
model = get_enzyme_constraint_model('iBsu1147_irr_enz_constraint_adj.json')

In [120]:
model.reactions.EX_glc__D_e.lower_bound = -2
model.optimize()
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e_reverse,0.001547,0,0.00%
fe3_e,EX_fe3_e_reverse,0.001664,0,0.00%
glc__D_e,EX_glc__D_e,2,6,28.57%
glc__D_e,EX_glc__D_e_reverse,5,6,71.43%
h_e,EX_h_e_reverse,4.049,0,0.00%
k_e,EX_k_e_reverse,0.341,0,0.00%
mg2_e,EX_mg2_e_reverse,0.04912,0,0.00%
nh4_e,EX_nh4_e_reverse,4.675,0,0.00%
o2_e,EX_o2_e,20.28,0,0.00%
pi_e,EX_pi_e_reverse,8.464,0,0.00%


In [154]:
# Merge split exchange reactions
for r in model.boundary:
    if r.id[-7:] == 'reverse':
        # print(r, r.upper_bound)
        r_forward = model.reactions.get_by_id(r.id[:-8])
        if r.upper_bound != 0:
            r_forward.lower_bound = -r.upper_bound
            # print(r_forward,r_forward.name, r_forward.bounds)
        r.remove_from_model()

In [70]:
model.optimize()
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.001105,0,0.00%
fe3_e,EX_fe3_e,0.001188,0,0.00%
glc__D_e,EX_glc__D_e,5,6,100.00%
h_e,EX_h_e,2.892,0,0.00%
k_e,EX_k_e,0.2435,0,0.00%
mg2_e,EX_mg2_e,0.03508,0,0.00%
nh4_e,EX_nh4_e,3.339,0,0.00%
o2_e,EX_o2_e,14.49,0,0.00%
pi_e,EX_pi_e,6.045,0,0.00%
so4_e,EX_so4_e,0.07338,0,0.00%


In [61]:
b_subtilis_N_per_OD = 2*1e8 #cells per mL at OD660 = 1 https://bionumbers.hms.harvard.edu/bionumber.aspx?s=n&v=4&id=105286
b_subtilis_weight = 2.2*1e-13
b_subtilis_N_per_OD*b_subtilis_weight*1000

0.044000000000000004

In [71]:
model.id = 'iBsu1147_irr_enz_constraint_adj'
model.name = 'iBsu1147_irr_enz_constraint_adj'

In [72]:
for m in model.metabolites:
    if not m.compartment in ['e', 'c']:
        m.compartment = m.id[-1]

In [73]:
model.compartments['c'] = 'Cytoplasm'
model.compartments['e'] = 'Extracellular'

In [74]:
model.compartments = {'c': 'Cytoplasm', 'e': 'Extracellular'}

In [152]:
# Fix very strict diphosphate kcat values by rmeoving it. Its rpesenve causes extreme phosphate flux
r = model.reactions.PPAm_num1
s = r.metabolites[model.metabolites.prot_pool]

r.add_metabolites({model.metabolites.prot_pool: -s})

In [153]:
cobra.io.write_sbml_model(model, 'iBsu1147_irr_enz_constraint_adj.xml')

In [151]:
model = cobra.io.read_sbml_model('iBsu1147_irr_enz_constraint_adj.xml')

In [150]:
with model:
    model.reactions.EX_glc__D_e.lower_bound = -9
    model.reactions.EX_ppi_e.upper_bound=1000
    solution = cobra.flux_analysis.pfba(model)
    print(model.summary(solution))
    model.metabolites.ppi_c.summary()
    r = model.reactions.PPAm_num1
    s = r.metabolites[model.metabolites.prot_pool]

    r.add_metabolites({model.metabolites.prot_pool: -s})
    print(model.reactions.PPAm_num1.reaction)
    solution = cobra.flux_analysis.pfba(model)
    print(model.summary(solution))

Objective
1.0 bio00006 = 0.6049834695400742

Uptake
------
Metabolite    Reaction     Flux  C-Number  C-Flux
     ca2_e    EX_ca2_e 0.001805         0   0.00%
     fe3_e    EX_fe3_e 0.001941         0   0.00%
  glc__D_e EX_glc__D_e        9         6 100.00%
       h_e      EX_h_e    1.871         0   0.00%
       k_e      EX_k_e   0.3978         0   0.00%
     mg2_e    EX_mg2_e  0.05732         0   0.00%
     nh4_e    EX_nh4_e    5.455         0   0.00%
      o2_e     EX_o2_e     22.9         0   0.00%
      pi_e     EX_pi_e    9.905         0   0.00%
     so4_e    EX_so4_e   0.1199         0   0.00%
 prot_pool enzyme_pool    0.165         0   0.00%

Secretion
---------
Metabolite Reaction   Flux  C-Number C-Flux
      ac_e  EX_ac_e -2.882         2 19.11%
     co2_e EX_co2_e -24.41         1 80.89%
     h2o_e EX_h2o_e -42.61         0  0.00%
     ppi_e EX_ppi_e -4.644         0  0.00%

h2o_c + ppi_c --> 2.0 h_c + 2.0 pi_c
Objective
1.0 bio00006 = 0.6082497114612208

Uptake
------
Met

In [100]:
with model:
    delta = 0.0001
    existing_flux = 0
    model.reactions.EX_glc__D_e.lower_bound = -3
    model.reactions.EX_gln__L_e.lower_bound = 0.04
    solution = model.optimize()
    print(model.summary())
    mets = ['g3p_c', 'dhap_c', '2pg_c', '3pg_c', 'gln__L_c']
    print(solution.shadow_prices[mets])
    m = model.metabolites.g3p_c
    try:
        r = model.reactions.get_by_id('DM_{0}'.format(m.id))
    except KeyError:
        r = model.add_boundary(m, type = 'demand')
    old_lb = r.lower_bound
    if old_lb != 0:
        print("existing DM with constraints for ", m.id)
        r.bounds = (old_lb + delta, 1000)
    else:
        r.bounds = (delta, 1000)
    if existing_flux:
        r_id, flux = existing_flux
        model.reactions.get_by_id(r_id).bounds = (flux, flux)
    sp = (model.slim_optimize()-solution.objective_value)/delta
    print(sp)

Objective
1.0 bio00006 = 0.21917074940770268

Uptake
------
Metabolite    Reaction      Flux  C-Number  C-Flux
     ca2_e    EX_ca2_e 0.0006538         0   0.00%
     fe3_e    EX_fe3_e 0.0007033         0   0.00%
  glc__D_e EX_glc__D_e         3         6 100.00%
       h_e      EX_h_e     1.631         0   0.00%
       k_e      EX_k_e    0.1441         0   0.00%
     mg2_e    EX_mg2_e   0.02076         0   0.00%
     nh4_e    EX_nh4_e     2.056         0   0.00%
      o2_e     EX_o2_e     8.641         0   0.00%
      pi_e     EX_pi_e     3.578         0   0.00%
     so4_e    EX_so4_e   0.04343         0   0.00%
 prot_pool enzyme_pool     0.165         0   0.00%

Secretion
---------
Metabolite    Reaction   Flux  C-Number C-Flux
     co2_e    EX_co2_e -9.167         1 97.86%
  gln__L_e EX_gln__L_e  -0.04         5  2.14%
     h2o_e    EX_h2o_e -15.88         0  0.00%
     ppi_e    EX_ppi_e -1.677         0  0.00%

g3p_c      -0.042653
dhap_c     -0.042653
2pg_c      -0.033897
3pg_c   

In [27]:
cobra.io.validate_sbml_model('iBsu1147_irr_enz_constraint_adj.xml')

(None,
 {'SBML_FATAL': [],
  'SBML_ERROR': ["E0 (Error): SBML component consistency (core, L16); Invalid attribute found on Compartment object; A Compartment object must have the required attributes 'id' and 'constant', and may have the optional attributes 'metaid', 'sboTerm', 'name', 'spatialDimensions', 'size' and 'units'. No other attributes from the SBML Level 3 Core namespace are permitted on a Compartment object.\nReference: L3V1 Section 4.5\n The required attribute 'id' is missing.\n",
   "E1 (Error): SBML component consistency (fbc, L6792); Chemical formula must be string; The value of attribute 'fbc:chemicalFormula' on the SBML <species> object must be set to a string consisting only of atomic names or user defined compounds and their occurrence.\nReference: L3V1 Fbc V3 Section 3.4\n Encountered '(' when expecting a capital letter. The chemicalFormula 'C13H22NO11PR2(C5H8O6PR)n' has incorrect syntax.\n",
   "E2 (Error): SBML component consistency (fbc, L8222); Chemical formula 